<a href="https://colab.research.google.com/github/gshrivathsan/demokct/blob/main/Hyperparameter_tuning_result1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
! pip install torch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0
! pip install datasets
! pip install rouge

In [ ]:
from datasets import load_dataset
multi_news = load_dataset("multi_news", split="test")

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

In [ ]:
multi_news.to_pandas()

,document,summary
0,GOP Eyes Gains As Voters In 11 States Pick Gov...,– It's a race for the governor's mansion in 11...
1,\n \n \n \n UPDATE: 4/19/2001 Read Richard Met...,– It turns out Facebook is only guilty of abou...
2,It's the Golden State's latest version of the ...,– Not a big fan of Southern California? Neithe...
3,The seed for this crawl was a list of every ho...,– Why did Microsoft buy Nokia's phone business...
4,After a year in which liberals scored impressi...,– The Supreme Court is facing a docket of high...
...,...,...
5617,Tweet with a location \n \n You can add locati...,– The traditional end-of-summit group photo at...
5618,Loic Venance/AFP/Getty Images \n \n The awards...,– Sofia Coppola scored a historic victory at t...
5619,(CNN) A federal criminal investigation into a ...,– The duck boat sinking that killed 17 on a Mi...
5620,An archive of the public statements deleted by...,– Note to tweeting politicians: Watch what you...


In [ ]:
multi_news = multi_news.train_test_split(test_size=0.2)

In [ ]:
! pip install transformers

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [ ]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(text=examples["summary"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_multi_news = multi_news.map(preprocess_function, batched=True)

Map:   0%|          | 0/4497 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model='t5-small')
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install accelerate>=0.20.1

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install accelerate -U

In [ ]:
!pip install accelerate -U

In [ ]:
!pip install torch

In [ ]:
!pip install transformers[torch]

In [ ]:
from transformers import Seq2SeqTrainingArguments,Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
output_dir="./results",
evaluation_strategy="epoch",
learning_rate=5e-5,
per_device_train_batch_size=8,
per_device_eval_batch_size=8,
weight_decay=0.01,
save_total_limit=5,
num_train_epochs=8,
fp16=False,
)

In [ ]:
trainer = Seq2SeqTrainer(
model=model,
args=training_args,
train_dataset=tokenized_multi_news["train"],
eval_dataset=tokenized_multi_news["test"],
tokenizer=tokenizer,
data_collator=data_collator,
)


In [ ]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,3.117700,2.748170
2,2.948400,2.706013
3,2.862800,2.689279
4,2.856500,2.671966


Epoch,Training Loss,Validation Loss
1,3.117700,2.748170
2,2.948400,2.706013
3,2.862800,2.689279
4,2.856500,2.671966
5,2.824900,2.668340
6,2.811500,2.662063
7,2.782500,2.660466
8,2.781400,2.658888


TrainOutput(global_step=4504, training_loss=2.8621190028334595, metrics={'train_runtime': 3804.6603, 'train_samples_per_second': 9.456, 'train_steps_per_second': 1.184, 'total_flos': 9737985359216640.0, 'train_loss': 2.8621190028334595, 'epoch': 8.0})

In [ ]:
document = "OpenAI, founded in December 2015, stands at the forefront of artificial intelligence research and development. The organization is driven by a noble mission to ensure that artificial general intelligence (AGI) benefits all of humanity. OpenAI conducts groundbreaking research in machine learning and natural language processing, producing transformative AI models such as GPT-3. These models have redefined the landscape of natural language understanding and generation, enabling a wide range of applications from language translation to question-answering. OpenAI places strong emphasis on ethical considerations, actively addressing bias and safety concerns, and strives to ensure that AGI is developed responsibly to avoid any harm.As an advocate for open-source collaboration, OpenAI generously shares its research and code with the AI community. The organization collaborates with various partners, institutions, and companies to drive innovation in the field. OpenAI offers commercial API access to its AI models, empowering developers to create diverse applications and services. In addition to its technological contributions, OpenAI engages in educational initiatives and public outreach to raise awareness about AI's transformative potential and its ethical and societal implications. OpenAI's work has left an indelible mark on the AI landscape, with a strong commitment to advancing AI research and fostering its responsible, equitable, and safe use for the benefit of humanity."
human_summary = """OpenAI, founded in 2015, leads AI research and champions ethical AI development for the benefit of humanity, sharing groundbreaking models like GPT-3. The organization combines open-source contributions with commercial accessibility, promoting innovation while emphasizing safety and responsible AI use"""

In [ ]:
def predict_summary(document):
  device = model.device
  tokenized = tokenizer([document], truncation =True, padding ='longest',return_tensors='pt')
  tokenized = {k: v.to(device) for k, v in tokenized.items()}
  tokenized_result = model.generate(**tokenized, max_length=128)
  tokenized_result = tokenized_result.to('cpu')
  predicted_summary = tokenizer.decode(tokenized_result[0])
  return predicted_summary

In [ ]:
predicted_summary = predict_summary(document)
predicted_summary

'<pad><extra_id_0>, and the AI community is a big part of the AI landscape. OpenAI, founded in December 2015, stands at the forefront of artificial intelligence research and development, and is driven by a noble mission to ensure that artificial general intelligence (AGI) benefits all of humanity. The organization is driven by a noble mission to ensure that artificial general intelligence (AGI) benefits all of humanity. OpenAI, founded in December 2015, stands at the forefront of artificial intelligence research and development, enabling AI models such as GPT-3. These models have redefined the landscape of language understanding and generation, enabling applications'

In [ ]:
from rouge import Rouge

def get_rouge_scores(actual_summary, predicted_summary):
    rouge = Rouge()
    scores = rouge.get_scores(predicted_summary, actual_summary)
    return [scores[0]['rouge-1']['f'], scores[0]['rouge-2']['f'], scores[0]['rouge-l']['f']]

In [ ]:
rouge_scores = get_rouge_scores(human_summary, predicted_summary)

# Print the ROUGE scores
print("ROUGE-1 F1 Score:", rouge_scores[0])
print("ROUGE-2 F1 Score:", rouge_scores[1])
print("ROUGE-L F1 Score:", rouge_scores[2])

ROUGE-1 F1 Score: 0.3058823480581315
ROUGE-2 F1 Score: 0.07619047157188236
ROUGE-L F1 Score: 0.3058823480581315
